In [1]:
import librosa
%matplotlib qt5
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import os
import librosa
import pandas as pd
import seaborn as sns
from IPython.display import Audio
from scipy.signal import find_peaks

In [2]:
FRAME_SIZE = 1024
HOP_LENGTH = 512

def amplitude_envelope(signal, frame_size, hop_length):
    """Calculate the amplitude envelope of a signal with a given frame size nad hop length."""
    amplitude_envelope = []

    # calculate amplitude envelope for each frame
    for i in range(0, len(signal), hop_length):
        amplitude_envelope_current_frame = max(signal[i:i+frame_size])
        amplitude_envelope.append(amplitude_envelope_current_frame)

    return np.array(amplitude_envelope)

In [3]:
audio = '/Users/borosabel/Documents/Uni/Thesis/PopMIR/Audio/west_coast/ dna_ kendrick lamar.mp3'

In [4]:
# Load the audio file
y, sr = librosa.load(audio)

# start_time = 5000
# end_time = start_time + 100000
# y = y[start_time:end_time]

In [5]:
# Plot the waveform
plt.figure(figsize=(14, 5))
librosa.display.waveshow(y, sr=sr)

# Provide labels and title
plt.title('Waveform')
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')

# Display the plot
plt.show()

In [12]:
onset_env = librosa.onset.onset_strength(y=y, sr=sr)

# Default delta value from librosa, modify it for your use case
delta = 0.5

# Detect onsets with the chosen delta
onsets = librosa.onset.onset_detect(onset_envelope=onset_env, sr=sr, delta=delta, backtrack=True)

# Convert frame indices to time
onset_times = librosa.frames_to_time(onsets, sr=sr)

# Prepare the plots
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(14, 10), sharex=True)

# Plot the waveform on the first subplot and mark the onsets
librosa.display.waveshow(y, sr=sr, ax=ax[0])
ax[0].vlines(onset_times, ymin=ax[0].get_ylim()[0], ymax=ax[0].get_ylim()[1], color='r', alpha=0.9, linestyle='--', label='Onsets')
ax[0].set(title='Waveform with Onsets')
ax[0].legend()

# Plot the onset strength on the second subplot
ax[1].plot(librosa.times_like(onset_env), onset_env, label='Onset Strength')
ax[1].set(title='Onset Strength')
ax[1].label_outer()  # Hides x-labels for all but the bottom plot
ax[1].legend()

plt.show()

In [13]:
tempo, beats = librosa.beat.beat_track(y=y, sr=sr, onset_envelope=onset_env)

# Convert the beat frames to timestamps
beat_times = librosa.frames_to_time(beats, sr=sr)

print(f"Estimated Tempo: {tempo} BPM")

Estimated Tempo: 135.99917763157896 BPM


In [12]:
import os
import pandas as pd
import librosa
import numpy as np

def compute_temporal_stability(beats, sr):
    beat_times = librosa.frames_to_time(beats, sr=sr)
    beat_intervals = np.diff(beat_times)
    stability = np.std(beat_intervals)
    return stability

def process_audio_files(folder_path):
    metrics = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".mp3"):  # Assuming the files are MP3 format
            file_path = os.path.join(folder_path, filename)
            print('Current file path:', file_path)
            # Load the audio file
            y, sr = librosa.load(file_path, sr=None)  # Use the native sampling rate

            # Compute the onset envelope and detect onsets
            onset_env = librosa.onset.onset_strength(y=y, sr=sr)
            onsets = librosa.onset.onset_detect(y=y, sr=sr, onset_envelope=onset_env, backtrack=True)
            onset_times = librosa.frames_to_time(onsets, sr=sr)

            # Compute tempo and beats
            tempo, beats = librosa.beat.beat_track(y=y, sr=sr, onset_envelope=onset_env)

            # Calculate temporal stability
            temporal_stability = compute_temporal_stability(beats, sr)

            # Append metrics for the current file to the list
            metrics.append({
                'Filename': filename,
                'Tempo': tempo,
                'Total Onsets': len(onset_times),
                'Onset Rate': len(onset_times) / (len(y) / sr),  # Total onsets divided by the duration of the audio in seconds
                'Temporal Stability': temporal_stability,
            })

    # Create a DataFrame from the metrics
    df = pd.DataFrame(metrics)
    return df

east_coast = '/Users/borosabel/Documents/Uni/Thesis/PopMIR/Audio/east_coast'
west_coast = '/Users/borosabel/Documents/Uni/Thesis/PopMIR/Audio/west_coast'

df1 = process_audio_files(east_coast)
df2 = process_audio_files(west_coast)

Current file path: /Users/borosabel/Documents/Uni/Thesis/PopMIR/Audio/east_coast/ pretty girls_ wale.mp3
Current file path: /Users/borosabel/Documents/Uni/Thesis/PopMIR/Audio/east_coast/ time 4 sum aksion_ redman.mp3
Current file path: /Users/borosabel/Documents/Uni/Thesis/PopMIR/Audio/east_coast/ what we do_ freeway feat jay z and beanie sigel.mp3
Current file path: /Users/borosabel/Documents/Uni/Thesis/PopMIR/Audio/east_coast/ who got da props_ black moon.mp3
Current file path: /Users/borosabel/Documents/Uni/Thesis/PopMIR/Audio/east_coast/ itty bitty piggy_ nicki minaj.mp3
Current file path: /Users/borosabel/Documents/Uni/Thesis/PopMIR/Audio/east_coast/ shot caller_ french montana feat charlie rock.mp3
Current file path: /Users/borosabel/Documents/Uni/Thesis/PopMIR/Audio/east_coast/ 5 oclock_ nonchalant.mp3
Current file path: /Users/borosabel/Documents/Uni/Thesis/PopMIR/Audio/east_coast/ just a friend_ biz markie.mp3
Current file path: /Users/borosabel/Documents/Uni/Thesis/PopMIR/Aud

In [13]:
df1

,Filename,Tempo,Total Onsets,Onset Rate,Temporal Stability
0,pretty girls_ wale.mp3,119.680851,714,2.750395,0.038107
1,time 4 sum aksion_ redman.mp3,104.166667,1203,5.571447,0.031565
2,what we do_ freeway feat jay z and beanie sig...,93.750000,1445,6.403679,0.018628
3,who got da props_ black moon.mp3,100.446429,1197,4.925755,0.013856
4,itty bitty piggy_ nicki minaj.mp3,92.213115,1110,4.496845,0.023270
...,...,...,...,...,...
95,ny state of mind_ nas.mp3,125.000000,1418,4.797939,0.040543
96,push it_ salt n pepa.mp3,130.813953,1148,4.340480,0.009336
97,come clean_ jeru the damaja.mp3,87.890625,1016,4.272369,0.035547
98,bodak yellow_ cardi b.mp3,125.000000,1608,6.880697,0.011244


In [14]:
df2

,Filename,Tempo,Total Onsets,Onset Rate,Temporal Stability
0,like whaaat_ problem feat bad luc.mp3,93.750000,1033,3.822123,0.032995
1,call it what u want_ above the law.mp3,98.684211,1256,4.521558,0.012759
2,la raza_ kid frost.mp3,108.173077,1145,4.045204,0.014563
3,dead homiez_ ice cube.mp3,112.500000,1062,4.805266,0.027877
4,6 in the mornin_ ice t.mp3,92.213115,906,4.043961,0.035895
...,...,...,...,...,...
95,posse on broadway_ sir mix a lot.mp3,86.538462,1496,5.319738,0.015024
96,gucci gucci_ kreayshawn.mp3,156.250000,765,4.008736,0.009111
97,bladadah_ mozzy.mp3,122.282609,1005,4.573061,0.042773
98,rest in piss_ brotha lynch hung.mp3,137.195122,1367,4.833281,0.013049


In [15]:
# Calculate average tempos
average_tempo_df1 = df1['Tempo'].mean()
average_tempo_df2 = df2['Tempo'].mean()

# Create a new DataFrame for plotting
tempo_data = {
    'Dataset': ['Dataset 1', 'Dataset 2'],
    'Average Tempo': [average_tempo_df1, average_tempo_df2]
}
average_tempo_df = pd.DataFrame(tempo_data)

# Plot using seaborn
plt.figure(figsize=(10, 6))
sns.barplot(x='Dataset', y='Average Tempo', data=average_tempo_df)
plt.title('Average Tempo Comparison')
plt.ylabel('Tempo (BPM)')
plt.xlabel('Dataset')

plt.show()

In [23]:
average_tempo_df1 = df1['Tempo'].mean()
average_tempo_df2 = df2['Tempo'].mean()
print(f'East Coast     vs    West Coast avg.Tempo\n{average_tempo_df1} - {average_tempo_df2}')

East Coast     vs    West Coast avg.Tempo
110.1164768291291 - 115.12923026478063


In [24]:
average_number_of_onset_df1 = df1['Onset Rate'].mean()
average_number_of_onset_df2 = df2['Onset Rate'].mean()
print(f'East Coast     vs    West Coast avg.Tempo\n{average_number_of_onset_df1} - {average_number_of_onset_df2}')

East Coast     vs    West Coast avg.Tempo
4.777115293474848 - 4.72658460262237


In [25]:
average_tempo_stability_df1 = df1['Temporal Stability'].mean()
average_tempo_stability_df2 = df2['Temporal Stability'].mean()
print(f'East Coast     vs    West Coast avg.Tempo\n{average_tempo_stability_df1} - {average_tempo_stability_df2}')

East Coast     vs    West Coast avg.Tempo
0.02641595873943965 - 0.020638804569234996
